In [1]:
import pandas as pd
import numpy as np
from itertools import combinations

pd.options.display.float_format = "{:.0f}".format

In [2]:
# output df.csv = main dataset used, total 4005 ted talks

df = pd.read_csv("raw_data/ted_talks_en.csv")

# df.to_csv("output/df.csv", index = False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4005 entries, 0 to 4004
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   talk_id         4005 non-null   int64  
 1   title           4005 non-null   object 
 2   speaker_1       4005 non-null   object 
 3   all_speakers    4001 non-null   object 
 4   occupations     3483 non-null   object 
 5   about_speakers  3502 non-null   object 
 6   views           4005 non-null   int64  
 7   recorded_date   4004 non-null   object 
 8   published_date  4005 non-null   object 
 9   event           4005 non-null   object 
 10  native_lang     4005 non-null   object 
 11  available_lang  4005 non-null   object 
 12  comments        3350 non-null   float64
 13  duration        4005 non-null   int64  
 14  topics          4005 non-null   object 
 15  related_talks   4005 non-null   object 
 16  url             4005 non-null   object 
 17  description     4005 non-null   o

In [3]:
# output df_topic.csv = split the list of topics of each ted talk into keywords, "topic_talkid" is the original talk_id, 31709 rows returned

list_topic = []
list_topic_talkid = []

for i in range(len(df)):
    topic_i = df["topics"][i].replace("[", "\'").replace("]", "\'").replace(", ", "\'").replace(" ", "_").split("\'")
    list_topic_i = [x for x in topic_i if x]
    list_topic_talkid_i = df["talk_id"][i].repeat(len(list_topic_i)).tolist()
    for j in range(len(list_topic_i)):
        list_topic.append(list_topic_i[j])
    for k in range(len(list_topic_talkid_i)):
        list_topic_talkid.append(list_topic_talkid_i[k])
        
df_topic = pd.DataFrame(
    {"topic": list_topic,
    "topic_talkid": list_topic_talkid})

# df_topic.to_csv("output/df_topic.csv", index = False)
df_topic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31709 entries, 0 to 31708
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   topic         31709 non-null  object
 1   topic_talkid  31709 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 495.6+ KB


In [4]:
# output df_topic_unique.csv = list of unique keywords in "topics", 458 results returned

list_topic_unique = df_topic["topic"].unique().tolist()
df_topic_unique = pd.DataFrame({"topic": list_topic_unique})

# df_topic_unique.to_csv("output/df_topic_unique.csv", index = False)
df_topic_unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458 entries, 0 to 457
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   topic   458 non-null    object
dtypes: object(1)
memory usage: 3.7+ KB


In [5]:
# unique keywords is then checked against the vocab list in "word2vec-google-news-300" (plz refer to google colab notebook), 380 results returned
# then read the result df_topic_unique_selected.csv generated from google colab

df_topic_unique_selected = pd.read_csv("colab_result/df_topic_unique_selected.csv")
df_topic_unique_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   topic   380 non-null    object
dtypes: object(1)
memory usage: 3.1+ KB


In [6]:
# output df_topic_final_views_avg.csv 
# "topic_count" count the total number of selected keywords in the main dataset, used as the size in scatter plot
# "views_avg" = "views" / "topic_count" ; "comments_avg" = "comments" / "topic_count" , used as the color in scatter plot

df_topic_final = df_topic_unique_selected.merge(df_topic, on = "topic", how = "left")
df_topic_final_freq = df_topic_final.groupby("topic").agg({"topic_talkid": pd.Series.nunique}).sort_values("topic_talkid", ascending=False)
df_topic_final_freq = df_topic_final_freq.rename(columns={"topic_talkid": "topic_count"})

df_topic_final_views = df_topic_final.merge(df, left_on = "topic_talkid", right_on = "talk_id", how = "left")[["topic","views","comments"]]
df_topic_final_views = df_topic_final_views.groupby("topic").agg({"views": np.sum, "comments": np.sum}).sort_values("views", ascending=False)

df_topic_final_views_avg = df_topic_final_views.join(df_topic_final_freq)
df_topic_final_views_avg["views_avg"] = df_topic_final_views_avg["views"]/df_topic_final_views_avg["topic_count"]
df_topic_final_views_avg["comments_avg"] = df_topic_final_views_avg["comments"]/df_topic_final_views_avg["topic_count"]
df_topic_final_views_avg = df_topic_final_views_avg.sort_values("views_avg", ascending=False)

# df_topic_final_views_avg.to_csv("output/df_topic_final_views_avg.csv", index = True)
df_topic_final_views_avg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 380 entries, introvert to gay
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   views         380 non-null    int64  
 1   comments      380 non-null    float64
 2   topic_count   380 non-null    int64  
 3   views_avg     380 non-null    float64
 4   comments_avg  380 non-null    float64
dtypes: float64(3), int64(2)
memory usage: 17.8+ KB


In [7]:
# generate the egdes list for keyword network, further plot by gephi
# 380 nodes & 104012 edges returned (some edges repeated in different ted talks)
# further calculate the average views and comments

df_edges = df_topic[df_topic["topic"].isin(df_topic_unique_selected["topic"])].sort_values(["topic_talkid","topic"])

list_edge = []
list_node1 = []
list_node2 = []
list_edge_talkid = []

for i in df_edges["topic_talkid"].unique().tolist(): #4003
    df_edge_i = df_edges[df_edges["topic_talkid"] == i]["topic"].tolist()
    y = list(combinations(df_edge_i, 2))
    for j in range(len(y)):
        list_edge.append(y[j])
        list_node1.append(y[j][0])
        list_node2.append(y[j][1])
        list_edge_talkid.append(i)

df_edge_talkid = pd.DataFrame(
    {"list_edge": list_edge,
    "node1": list_node1,
    "node2": list_node2,
    "edge_talkid": list_edge_talkid})

df_edge_talkid = df_edge_talkid.merge(df[["talk_id","views","comments"]], left_on = "edge_talkid", right_on = "talk_id", how = "left")
df_edge_talkid = df_edge_talkid.drop(columns="edge_talkid")

# df_edge_talkid.to_csv("output/df_edge_talkid.csv", index = False)
df_edge_talkid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104012 entries, 0 to 104011
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   list_edge  104012 non-null  object 
 1   node1      104012 non-null  object 
 2   node2      104012 non-null  object 
 3   talk_id    104012 non-null  int64  
 4   views      104012 non-null  int64  
 5   comments   90226 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 5.6+ MB
